In [1]:
import os
import torch
from torch import nn
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch.optim import Adam
from collections import defaultdict
import numpy.ma as ma
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from nn.models import DWSModel
from typing import Tuple, NamedTuple
from nn.layers import BN, DWSLayer,InvariantLayer, Dropout, ReLU
from nn.layers.base import BaseLayer,GeneralSetLayer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.backends.cudnn.enabled = False

cpu


In [2]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleMLP, self).__init__()
        self.hidden1 = nn.Linear(input_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        return self.output(x)

def permute_layer_weights(layer, perm):
    """Permutes weights and biases of a layer based on the given permutation matrix."""
    weight = layer.weight.data.clone()
    bias = layer.bias.data.clone()

    # Permute rows of weights (output dimension)
    layer.weight.data = weight[perm, :]

    # Permute bias (output dimension)
    layer.bias.data = bias[perm]
    return layer

def permute_model(model, permutations):
    """Applies a list of permutations to a model's layers."""
    permute_layer_weights(model.hidden1, permutations[0])
    permute_layer_weights(model.hidden2, permutations[1])

    # Adjust input weights of the second hidden layer
    model.hidden2.weight.data = model.hidden2.weight.data[:, permutations[0]]
    # Adjust input weights of the output layer
    model.output.weight.data = model.output.weight.data[:, permutations[1]]
    return model

def generate_permutation_matrix(hidden_dim):
    """Generates a random permutation matrix."""
    perm = np.random.permutation(hidden_dim)
    return torch.tensor(perm, dtype=torch.long)

# Example Usage
input_dim = 10
hidden_dim = 16
output_dim = 5

# Initialize two identical models
model_a = SimpleMLP(input_dim, hidden_dim, output_dim)
model_b = SimpleMLP(input_dim, hidden_dim, output_dim)
model_b.load_state_dict(model_a.state_dict())  # Ensure identical initial weights

# Generate random permutations for each hidden layer
perm1 = generate_permutation_matrix(hidden_dim)
perm2 = generate_permutation_matrix(hidden_dim)

# Apply permutations to model B
model_c = permute_model(model_b, [perm1, perm2])

# Verify functionality is preserved
x = torch.rand(1, input_dim)
output_a = model_a(x)
output_b = model_b(x)
# output_c = model_c(x)

print("Outputs are identical:", torch.allclose(output_a, output_b))

Outputs are identical: True
